## Setup

In [ ]:
%pip install 'vanna[openai]'

In [1]:
from vanna.openai import OpenAI_Chat
from vanna.vannadb import VannaDB_VectorStore


In [6]:



class MyVanna(VannaDB_VectorStore, OpenAI_Chat):
    def __init__(self, config=None):
        MY_VANNA_MODEL = 'gpt_ral_model'
        VannaDB_VectorStore.__init__(self, vanna_model=MY_VANNA_MODEL, vanna_api_key='', config=config)
        OpenAI_Chat.__init__(self, config=config)

vn = MyVanna(config={'api_key': 's', 'model': 'gpt-4-turbo'})


In [7]:
vn.connect_to_sqlite('sakila_master.db')

## Training
You only need to train once. Do not train again unless you want to add more training data.

In [5]:

df_ddl = vn.run_sql("SELECT type, sql FROM sqlite_master WHERE sql is not null")

for ddl in df_ddl['sql'].to_list():
  vn.train(ddl=ddl)


Adding ddl: CREATE TABLE actor (
  actor_id numeric NOT NULL ,
  first_name VARCHAR(45) NOT NULL,
  last_name VARCHAR(45) NOT NULL,
  last_update TIMESTAMP NOT NULL,
  PRIMARY KEY  (actor_id)
  )
Adding ddl: CREATE INDEX idx_actor_last_name ON actor(last_name)
Adding ddl: CREATE TRIGGER actor_trigger_ai AFTER INSERT ON actor
 BEGIN
  UPDATE actor SET last_update = DATETIME('NOW')  WHERE rowid = new.rowid;
 END
Adding ddl: CREATE TRIGGER actor_trigger_au AFTER UPDATE ON actor
 BEGIN
  UPDATE actor SET last_update = DATETIME('NOW')  WHERE rowid = new.rowid;
 END
Adding ddl: CREATE TABLE country (
  country_id SMALLINT NOT NULL,
  country VARCHAR(50) NOT NULL,
  last_update TIMESTAMP,
  PRIMARY KEY  (country_id)
)
Adding ddl: CREATE TRIGGER country_trigger_ai AFTER INSERT ON country
 BEGIN
  UPDATE country SET last_update = DATETIME('NOW')  WHERE rowid = new.rowid;
 END
Adding ddl: CREATE TRIGGER country_trigger_au AFTER UPDATE ON country
 BEGIN
  UPDATE country SET last_update = DATETIME

In [6]:

# The following are methods for adding training data. Make sure you modify the examples to match your database.

# DDL statements are powerful because they specify table names, colume names, types, and potentially relationships
vn.train(ddl="""
    CREATE TABLE IF NOT EXISTS my-table (
        id INT PRIMARY KEY,
        name VARCHAR(100),
        age INT
    )
""")

# Sometimes you may want to add documentation about your business terminology or definitions.
vn.train(documentation="Our business defines OTIF score as the percentage of orders that are delivered on time and in full")

# You can also add SQL queries to your training data. This is useful if you have some queries already laying around. You can just copy and paste those from your editor to begin generating new SQL.
vn.train(sql="SELECT * FROM my-table WHERE name = 'John Doe'")


Adding ddl: 
    CREATE TABLE IF NOT EXISTS my-table (
        id INT PRIMARY KEY,
        name VARCHAR(100),
        age INT
    )
Adding documentation....
Using model gpt-4-turbo for 65.0 tokens (approx)
Question generated with sql: What are all the details for the entry where the name is 'John Doe'? 
Adding SQL...


'867ca04a7c5b8f25b34f3f9c1aaaa6b1-sql'

In [7]:
# At any time you can inspect what training data the package is able to reference
training_data = vn.get_training_data()
training_data

,id,training_data_type,question,content
0,162717-ddl,ddl,None,CREATE INDEX idx_rental_fk_staff_id ON rental(...
1,162713-ddl,ddl,None,CREATE TRIGGER payment_trigger_au AFTER UPDATE...
2,162687-ddl,ddl,None,CREATE TRIGGER film_actor_trigger_au AFTER UPD...
3,162679-ddl,ddl,None,CREATE INDEX idx_fk_language_id ON film(langua...
4,162683-ddl,ddl,None,CREATE TABLE film_actor (\n actor_id INT NOT ...
...,...,...,...,...
73,162673-ddl,ddl,None,CREATE INDEX idx_customer_fk_address_id ON cus...
74,162700-ddl,ddl,None,CREATE INDEX idx_fk_staff_store_id ON staff(st...
75,162694-ddl,ddl,None,CREATE TABLE inventory (\n inventory_id INT N...
76,162675-ddl,ddl,None,CREATE INDEX idx_customer_last_name ON custome...


In [ ]:
# You can remove training data if there's obsolete/incorrect information. 
vn.remove_training_data(id='1-ddl')


## Asking the AI
Whenever you ask a new question, it will find the 10 most relevant pieces of training data and use it as part of the LLM prompt to generate the SQL.

In [8]:
vn.ask(question="How many cities are there")

HTTPSConnectionPool(host='ask.vanna.ai', port=443): Max retries exceeded with url: /rpc (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x76ed9a135d80>: Failed to establish a new connection: [Errno 101] Network is unreachable'))


(None, None, None)

In [ ]:
from vanna.flask import VannaFlaskApp
app = VannaFlaskApp(vn)
app.run(port="8098")

 * Serving Flask app 'vanna.flask'
 * Debug mode: off
Using model gpt-4-turbo for 674.5 tokens (approx)


[2024-05-23 14:35:38,832] ERROR in app: Exception on /api/v0/generate_sql [GET]
Traceback (most recent call last):
  File "/home/mohamedali/anaconda3/envs/TEXT-RAG2SQL/lib/python3.10/site-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
  File "/home/mohamedali/anaconda3/envs/TEXT-RAG2SQL/lib/python3.10/site-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/home/mohamedali/anaconda3/envs/TEXT-RAG2SQL/lib/python3.10/site-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
OSError: [Errno 101] Network is unreachable

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/mohamedali/anaconda3/envs/TEXT-RAG2SQL/lib/python3.10/site-packages/urllib3/connectionpool.py", line 793, in urlopen
    response = self._make_request(
  File "/home/mohamedali/anaconda3/envs/TEXT-RAG2SQL/lib/python3.10/site-packages/